# 에이전트 평가(Evaluating Agents)

이메일 어시스턴트는 라우터를 사용하여 이메일을 분류한 후 응답 생성을 위해 에이전트에게 전달합니다. 실제 운영 환경에서 제대로 작동할지 어떻게 확신할 수 있을까요? 그래서 테스트가 중요한 이유입니다: 응답 품질, 토큰 사용량, 지연 시간, 분류 정확도 같은 정량적 지표를 통해 에이전트 아키텍처에 대한 결정을 안내합니다. 

[LangSmith](https://docs.smith.langchain.com/)는 에이전트 테스트를 위한 두 가지 주요 방법을 제공합니다. 

![overview-img](assets/overview_eval.png)

## Resources

- Notebook Reference: [evaluation.ipynb](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/evaluation.ipynb)
- For running evaluations: [test_tools.py](https://github.com/langchain-ai/agents-from-scratch/blob/main/notebooks/test_tools.py), [/tests/](https://github.com/langchain-ai/agents-from-scratch/tree/main/tests)
- For LangSmith Studio: [src/email_assistant](https://github.com/langchain-ai/agents-from-scratch/tree/main/src/email_assistant)
- Slides: [Building Ambient Agents with LangGraph - Building Agents & Evaluations.pdf](https://files.cdn.thinkific.com/file_uploads/967498/attachments/5f6/a6b/958/Building_Ambient_Agents_with_LangGraph_-_Building_Agents___Evaluations.pdf)

## 환경 설정

In [1]:
from dotenv import load_dotenv


load_dotenv("../../.env", override=True)

True

## 평가 실행 방법

#### Pytest / Vitest

[Pytest](https://docs.pytest.org/en/stable/)와 Vitest는 Python 및 JavaScript 에서 테스트를 작성하기 위한 강력한 도구입니다. LangSmith는 이러한 프레임워크와 통합되어, 결과를 LangSmith에 기록하는 테스트를 작성하고 실행할 수 있도록 지원합니다. 이 노트북에서는 Pytest를 사용합니다.
* Pytest는 해당 프레임워크에 이미 익숙한 개발자가 시작하기에 좋은 방법입니다.
* Pytest는 각 에이전트 테스트 케이스에 일반화하기 어려운 특정 검증과 성공 기준이 필요한 더 복잡한 평가에 유용합니다.

#### LangSmith 데이터셋

LangSmith에서 데이터셋을 생성하고 LangSmith 평가 API를 사용하여 해당 데이터셋에 대해 어시스턴트를 실행할 수 있습니다.
* LangSmith 데이터셋은 테스트 스위트를 협력하여 구축하는 팀에게 유용합니다.
* 운영 트레이스, 어노테이션 큐, 합성 데이터 생성 등을 활용하여 계속해서 확장되는 골든 데이터셋에 예제를 추가할 수 있습니다.
* LangSmith 데이터셋은 데이터셋의 모든 테스트 케이스에 적용할 수 있는 평가자(예: 유사도, 정확히 일치 정확도 등)를 정의할 수 있을 때 특히 유용합니다.

## 테스트 케이스

테스트는 종종 테스트 케이스를 정의하는 것에서 시작하며, 이는 어려운 과정일 수 있습니다. 여기서는 처리하고자 하는 예시 이메일 모음과 테스트할 몇 가지 사항을 `eval/email_dataset.py` 파일에 다음과 같이 정의했습니다.

1.  **Input Emails**: 다양한 이메일 예시 모음
2.  **Ground Truth Classifications**: `Respond`, `Notify`, `Ignore`
3.  **Expected Tool Calls**: 응답이 필요한 각 이메일에 대해 호출되는 도구
4.  **Response Criteria**: 회신이 필요한 이메일에 대한 좋은 응답의 조건

참고로, 우리는 다음과 같은 두 종류의 테스트를 모두 가지고 있습니다.
-   엔드-투-엔드 "통합" 테스트 (예: 입력 이메일 -> 에이전트 -> 최종 결과물 vs 응답 기준)
-   워크플로우의 특정 단계를 위한 테스트 (예: 입력 이메일 -> 에이전트 -> 분류 vs 정답 분류)

In [2]:
from email_assistant.eval.email_dataset import (
    email_inputs,
    expected_tool_calls,
    response_criteria_list,
    triage_outputs_list,
)


test_case_ix = 0

print("Email Input:", email_inputs[test_case_ix])
print("Expected Triage Output:", triage_outputs_list[test_case_ix])
print("Expected Tool Calls:", expected_tool_calls[test_case_ix])
print("Response Criteria:", response_criteria_list[test_case_ix])

Email Input: {'author': '앨리스 스미스 &lt;alice.smith@company.com&gt;', 'to': '랜스 마틴 &lt;lance@company.com&gt;', 'subject': 'API 문서에 대한 간단한 질문', 'email_thread': '안녕하세요, 랜스님,\n\n새로운 인증 서비스의 API 문서를 검토하던 중, 몇몇 엔드포인트가 사양에서 누락된 것을 발견했습니다. 이것이 의도된 것인지, 아니면 문서를 업데이트해야 하는지 명확히 설명해주실 수 있나요?\n\n특히 다음 엔드포인트에 대해 문의드립니다:\n- /auth/refresh\n- /auth/validate\n\n감사합니다!\n앨리스 드림'}
Expected Triage Output: respond
Expected Tool Calls: ['write_email', 'done']
Response Criteria: 
• write_email 도구 호출을 사용하여 이메일을 보내 질문을 인지했음을 알리고 조사가 진행될 것임을 확인합니다.



## Pytest 예제

Pytest를 사용하여 워크플로우의 특정 부분에 대한 테스트를 작성하는 방법을 알아봅니다. 

`email_assistant`가 이메일에 응답할 때 올바른 도구 호출을 하는지 테스트합니다.

In [3]:
from email_assistant.email_assistant import email_assistant
from email_assistant.eval.email_dataset import email_inputs, expected_tool_calls
from email_assistant.utils import extract_tool_calls, format_messages_string
from langsmith import testing as t
import pytest


@pytest.mark.langsmith
@pytest.mark.parametrize(
    ("email_input", "expected_calls"),
    [
        # 이메일 답장이 예상되는 몇 가지 예시를 선택하세요
        (email_inputs[0], expected_tool_calls[0]),
        (email_inputs[3], expected_tool_calls[3]),
    ],
)
def test_email_dataset_tool_calls(email_input, expected_calls):
    """
    이메일 처리 과정에서 예상되는 도구 호출이 포함되는지 테스트합니다.

    이 테스트는 이메일 처리 중 모든 예상 도구가 호출되는지 확인하지만,
    도구 호출 순서나 도구별 호출 횟수는 검사하지 않습니다.
    필요 시 이러한 측면에 대한 추가 검사를 포함시킬 수 있습니다.
    """
    # 이메일 어시스턴트를 실행합니다.
    messages = [("user", str(email_input))]
    result = email_assistant.invoke({"messages": messages})

    # 메시지 목록에서 도구 호출 추출합니다.
    extracted_tool_calls = extract_tool_calls(result["messages"])

    # 추출된 도구 호출에 예상된 모든 호출이 포함되어 있는지 확인합니다.
    missing_calls = [
        call for call in expected_calls if call.lower() not in extract_tool_calls
    ]

    t.log_outputs(
        {
            "missing_calls": missing_calls,
            "extracted_tool_calls": extracted_tool_calls,
            "response": format_messages_string(result["messages"]),
        }
    )

    # 예상된 호출이 누락되지 않으면 테스트가 통과됩니다.
    assert len(missing_calls) == 0

- [Pytest로 실행하고 테스트 결과를 LangSmith에 로그로 남기기](https://langchain-docs.jeongsk.work/langsmith/pytest) 위해서는, 함수에 `@pytest.mark.langsmith` 데코레이터만 추가하면 됩니다. 이렇게 하면 테스트 결과가 LangSmith에 로그로 기록됩니다.
- [`@pytest.mark.parametrize`](https://langchain-docs.jeongsk.work/langsmith/pytest#parametrize-with-pytest-mark-parametrize)를 통해 데이터셋 예제를 테스트 함수에 전달할 수 있습니다.

#### Pytest 실행하기

커맨드 라인에서 테스트를 실행할 수 있습니다. 위의 코드를 파이썬 파일로 정의했습니다. 프로젝트 루트에서 다음 명령어를 실행하세요:

```shell
LANGSMITH_TEST_SUITE='Email assistant: Test Tools For Interrupt' pytest test_tools.py
```

#### 실험 결과 보기

LangSmith UI에서 결과를 확인할 수 있습니다.

`assert len(missing_calls) == 0`은 LangSmith의 `Pass` 열에 기록됩니다. 

`log_outputs`는 `Outputs` 열로 전달되며, 함수 인수는 `Inputs` 열로 전달됩니다. `@pytest.mark.parametrize`로 전달된 각 입력은 LangSmith의 `LANGSMITH_TEST_SUITE` 프로젝트 이름 아래 별도의 행으로 기록되며, 이는 `Datasets & Experiments`에서 확인할 수 있습니다.

![Test Results](./assets/test_result.png)

## LangSmith 데이터셋 예제

![overview-img](./assets/eval_detail.png)

LangSmith 데이터셋으로 평가를 실행하는 방법을 살펴봅니다. 이전 Pytest 예제에서는 이메일 어시스턴트의 도구 호출 정확도를 평가했습니다. 이번에 평가할 데이터셋은 이메일 어시스턴트의 분류(triage) 단계에 특화되어 있으며, 이메일에 응답이 필요한지 여부를 분류하는 것입니다.

#### 데이터셋 정의

LangSmith SDK를 사용하여 [LangSmith에서 데이터셋을 생성](https://langchain-docs.jeongsk.work/langsmith/manage-datasets-programmatically#create-a-dataset)할 수 있습니다. 아래 코드는 `eval/email_dataset.py` 파일에 있는 테스트 케이스들로 데이터셋을 생성합니다.

In [4]:
from email_assistant.eval.email_dataset import examples_triage
from langsmith import Client


# LangSmith 클라이언트 초기화
client = Client()

# 데이터셋 이름
dataset_name = "E-mail Triage Evaluation"

# 데이터셋이 존재하지 않으면 신규로 생성합니다.
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description="이메일과 분류 결정에 대한 데이터셋입니다.",
    )

    # 데이터셋에 예제 추가
    client.create_examples(
        dataset_id=dataset.id,
        examples=examples_triage,
    )

#### 대상 함수

데이터셋은 다음과 같은 구조를 가지고 있으며, 이메일 입력과 이메일에 대한 정답(ground truth) 분류 결과를 출력으로 포함합니다:

```json
examples_triage = [
  {
      "inputs": {"email_input": email_input_1},
      "outputs": {"classification": triage_output_1},   # 참고: 이것은 생성된 데이터셋에서 reference_output이 됩니다
  }, ...
]
```

In [5]:
from pprint import pprint


print("데이터셋 예제 입력 (inputs):")
pprint(examples_triage[0]["inputs"])
print()
print("데이터셋 예제 참조 출력 (reference_outputs):")
pprint(examples_triage[0]["outputs"])

데이터셋 예제 입력 (inputs):
{'email_input': {'author': '앨리스 스미스 &lt;alice.smith@company.com&gt;',
                 'email_thread': '안녕하세요, 랜스님,\n'
                                 '\n'
                                 '새로운 인증 서비스의 API 문서를 검토하던 중, 몇몇 엔드포인트가 사양에서 '
                                 '누락된 것을 발견했습니다. 이것이 의도된 것인지, 아니면 문서를 업데이트해야 '
                                 '하는지 명확히 설명해주실 수 있나요?\n'
                                 '\n'
                                 '특히 다음 엔드포인트에 대해 문의드립니다:\n'
                                 '- /auth/refresh\n'
                                 '- /auth/validate\n'
                                 '\n'
                                 '감사합니다!\n'
                                 '앨리스 드림',
                 'subject': 'API 문서에 대한 간단한 질문',
                 'to': '랜스 마틴 &lt;lance@company.com&gt;'}}

데이터셋 예제 참조 출력 (reference_outputs):
{'classification': 'respond'}


- 데이터셋 입력을 받아 이메일 어시스턴트에 전달하는 함수를 정의합니다. 
- LangSmith [evaluate API](https://langchain-docs.jeongsk.work/langsmith/evaluation-concepts)는 `inputs` 딕셔너리를 이 함수에 전달합니다. 
- 그러면 이 함수는 에이전트의 출력이 담긴 딕셔너리를 반환합니다. 
- 분류(triage) 단계를 평가하고 있기 때문에, 분류 결정만 반환하면 됩니다.

In [6]:
email_assistant.nodes["triage_router"].invoke(
    {
        "email_input": examples_triage[0]["inputs"]["email_input"],
    }
)

📧 Classification: RESPOND - This email requires a response


Command(update={'classification_decision': 'respond', 'messages': [{'role': 'user', 'content': 'Respond to the email: \n\n**Subject**: API 문서에 대한 간단한 질문\n**From**: 앨리스 스미스 &lt;alice.smith@company.com&gt;\n**To**: 랜스 마틴 &lt;lance@company.com&gt;\n\n안녕하세요, 랜스님,\n\n새로운 인증 서비스의 API 문서를 검토하던 중, 몇몇 엔드포인트가 사양에서 누락된 것을 발견했습니다. 이것이 의도된 것인지, 아니면 문서를 업데이트해야 하는지 명확히 설명해주실 수 있나요?\n\n특히 다음 엔드포인트에 대해 문의드립니다:\n- /auth/refresh\n- /auth/validate\n\n감사합니다!\n앨리스 드림\n\n---\n'}]}, goto='response_agent')

In [7]:
def target_email_assistant(inputs: dict) -> dict:
    """워크플로우 기반 이메일 어시스턴트를 통해 이메일을 처리합니다."""

    # 이메일 어시스턴트에서 triage_router 노드를 가져옴
    triage_router = email_assistant.nodes["triage_router"]

    # 입력된 이메일을 triage_router에 전달하여 분류 작업 수행
    response = triage_router.invoke(
        {
            "email_input": inputs["email_input"],
        }
    )

    # 분류 결정 결과를 반환
    # 분류 결정 결과는 Command 객체의 update에 dict 형태로 담겨있음
    return {
        "classification_decision": response.update["classification_decision"],
    }

#### 평가자 함수

이제 평가자 함수를 생성합니다. 무엇을 평가하고 싶을까요? 데이터셋에는 참조 출력이 있고, 위의 함수에서 정의된 에이전트 출력이 있습니다.

* 참조 출력: `"reference_outputs": {"classification": triage_output_1} ...`
* 에이전트 출력: `"outputs": {"classification_decision": agent_output_1} ...`

에이전트의 출력이 참조 출력과 일치하는지 평가하고 싶습니다. 따라서 두 값을 비교하는 평가자 함수만 있으면 됩니다. 여기서 `outputs`는 에이전트의 출력이고 `reference_outputs`는 데이터셋의 참조 출력입니다.

In [8]:
def classification_evaluator(
    outputs: dict,
    reference_outputs: dict,
) -> bool:
    """답변이 예상 답변과 정확히 일치하는지 확인합니다."""

    # 에이전트의 분류 결정과 참조 분류 결과를 비교
    result: bool = (
        outputs["classification_decision"].lower()
        == reference_outputs["classification"].lower()
    )

    return result

### Running Evaluation

Now, the question is: how are these things hooked together? The evaluate API takes care of it for us. It passes the `inputs` dict from our dataset the target function. It passes the `reference_outputs` dict from our dataset to the evaluator function. And it passes the `outputs` of our agent to the evaluator function. 

Note this is similar to what we did with Pytest: in Pytest, we passed in the dataset example inputs and reference outputs to the test function with `@pytest.mark.parametrize`.